# PandasDataFrameOutputParser

- 著者: [Jiwon Kim](https://github.com/brian604)
- ピアレビュー: [Junseong Kim](https://www.linkedin.com/in/%EC%A4%80%EC%84%B1-%EA%B9%80-591b351b2/), [Jaehun Choi](https://github.com/ash-hun)
- 校正: [Two-Jay](https://github.com/Two-Jay)
- これは [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial) の一部です

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/03-OutputParser/05-PandasDataFrameOutputParser.ipynb)[![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/03-OutputParser/05-PandasDataFrameOutputParser.ipynb)

## 概要

このチュートリアルでは、LLMの出力を```pd.DataFrame```形式で制御する方法を扱います。

Pandasは、表形式データ(テーブル形式)の処理に便利なパッケージで、データサイエンティストに広く使用されています。データの探索、クリーニング、処理に役立ちます。

```pd.DataFrame```とその機能について詳しく学ぶには、Pandasの公式チュートリアル、
[10 minutes to pandas](https://pandas.pydata.org/docs/user_guide/10min.html)をご覧ください。

```PandasDataFrameOutputParser```は、[公式APIドキュメント](https://python.langchain.com/api_reference/langchain/output_parsers/langchain.output_parsers.pandas_dataframe.PandasDataFrameOutputParser.html)によると、Pandas DataFrame形式を使用して出力を解析します。
- これは、LLMクエリへの構造化出力の形成と、LLMクエリへの構造化入力の形成の両方に利用できます
    - 出力として、[01-PydanticOutputParser](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/03-OutputParser/01-PydanticOutputParser.ipynb)やこの学習ガイドの他の箇所でpydanticの使用法を詳しく見ることができます
    - 入力として、```pd.DataFrame```データセットを利用して、LLMがデータと対話できるようにします

**このチュートリアルの目的**

- ```PandasDataFrameOutputParser```が```pd.DataFrame```と対話するために使用される場所を知る

**データ取得**
- [figshare.com](https://figshare.com)にアクセスして、CSVからPDFまでの学術研究の成果を発見できます。

### 目次

- [概要](#概要)
- [環境設定](#環境設定)
- [モデルを使用しない場合](#モデルを使用しない場合)
- [モデルを使用する場合](#モデルを使用する場合)

### 参考文献

- [PandasDataFrameOutputParser公式API](https://python.langchain.com/api_reference/langchain/output_parsers/langchain.output_parsers.pandas_dataframe.PandasDataFrameOutputParser.html)
- [10 minutes to pandas](https://pandas.pydata.org/docs/user_guide/10min.html)

内部参照
- [01-PydanticOutputParser](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/03-OutputParser/01-PydanticOutputParser.ipynb)

## 環境設定

環境をセットアップします。詳細については[環境設定](https://wikidocs.net/257836)を参照してください。

**[注意]**
- ```langchain-opentutorial```は、チュートリアル用の使いやすい環境設定、便利な関数、ユーティリティを提供するパッケージです。
- 詳細については[```langchain-opentutorial```](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi)をご確認ください。

In [1]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [2]:
# pprintppとtypingのインストール
%pip install pprintpp
%pip install typing

In [3]:
# 必要なパッケージをインストール
from langchain_opentutorial import package

package.install(
    [
        "pandas",
        "langchain",
        "langchain_core",
        "langchain_community",
        "langchain_openai",
    ],
    verbose=False,
    upgrade=False,
)

In [4]:
# 環境変数を設定
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "PandasDataFrameOutputParser",
    }
)

環境変数が正常に設定されました。


あるいは、```OPENAI_API_KEY```などのAPIキーを```.env```ファイルに設定して読み込むこともできます。

[注意] 前の手順で必要なAPIキーを既に設定している場合、これは不要です。

In [5]:
# .envファイルからAPIキーを読み込む
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [6]:
import pprint
from typing import Any, Dict
import os
import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

print(f'{os.environ["LANGCHAIN_PROJECT"]}プロジェクトがlangsmithエンドポイントで追跡されています')

LANGCHAIN_PROJECTプロジェクトがlangsmithエンドポイントで追跡されています


In [7]:
# Langchain OpenAIモデルを初期化（gpt-4o-miniの使用を推奨）
model = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

## モデルを使用しない場合
- ```format_parser_output```関数は、パーサー出力を辞書形式に変換し、出力をフォーマットするために使用されます。
- 入力が```pd.DataFrame```またはファイルの場合。

In [8]:
# 目的: 出力のみ。ヘルパー関数です。
def format_parser_output(parser_output: Dict[str, Any]) -> None:
    length_val = len(list(parser_output.values()))
    if length_val == 1 and isinstance(parser_output.values, (int, float)):
        return {"result": parser_output}
    else:
        # パーサー出力(dict)の「キー」を反復処理
        for key in parser_output.keys():
            parser_output[key] = parser_output[key].to_dict()
        # Pretty Print
        return pprint.PrettyPrinter(width=4, compact=True).pprint(parser_output)

- チュートリアルで何度も取り上げられた「典型的な」```titanic.csv```データは退屈なので
- [figshare.com](https://figshare.com)から取得しましょう
    - データリンク
        - 永続的: [Air-Traffic-Passenger-Statistics-csv.csv](https://doi.org/10.6084/m9.figshare.24781905.v1)
        - ウェブサイト: [Air-Traffic-Passenger-Statistics-csv.csv](https://figshare.com/articles/dataset/Air-Traffic-Passenger-Statistics-csv_csv/24781905?file=43579620)

In [9]:
# DataFrameを定義
df = pd.read_csv("./data/Air-Traffic-Passenger-Statistics-csv.csv")
df.head()

Activity Period  ...  data_loaded_at
0           199907  ...  2023/11/20 07:02:25 AM
...

In [10]:
# 列名をクリーンアップ
df.columns = df.columns.str.strip().str.replace('"', "").str.replace("'", "")

In [11]:
# パーサーを設定し、プロンプトテンプレートに指示を注入
parser = PandasDataFrameOutputParser(dataframe=df)

# パーサーの指示を出力
print(parser.get_format_instructions())

出力は、操作を文字列としてフォーマットし、その後にコロン、クエリする列または行、オプションの配列パラメータを続けます。
1. 列名は以下の可能な列に制限されます。
2. 配列は、[1,3,5]のようにカンマ区切りの数値リスト、または[0..4]のような数値範囲である必要があります。
3. 配列はオプションであり、必ずしも必要ではないことを覚えておいてください。
4. 列が可能な列にない場合、または操作が有効なPandas DataFrame操作でない場合は、「Invalid column」または「Invalid operation」で始まる文として理由を返します。

例として、以下の形式:
1. 文字列「column:num_legs」は、num_legsが可能な列である場合に列num_legsを取得する正しくフォーマットされたインスタンスです。
2. 文字列「row:1」は、行1を取得する正しくフォーマットされたインスタンスです。
3. 文字列「column:num_legs[1,2]」は、num_legsが可能な列である場合に行1と2の列num_legsを取得する正しくフォーマットされたインスタンスです。

以下は可能な列です:
```
Activity Period, Activity Period Start Date, Operating Airline, ...
```


- 列の値を検索する例

In [12]:
print(df["GEO Region"].head())

0        US
1        US
2        US
3    Europe
4    Europe
Name: GEO Region, dtype: object


In [13]:
# 列を操作する例
df_query = "column:GEO Region[0..4]"
output = parser.parse(df_query)
format_parser_output(output)

{'GEO Region': {0: 'US', 1: 'US', 2: 'US', 3: 'Europe', 4: 'Europe'}}


- 行検索の例

In [14]:
# 最初の行を取得する例
df_query = "row:1"
output = parser.parse(df_query)
format_parser_output(output)

{'1': {'Activity Period': 199907,
       'Operating Airline': 'ATA Airlines',
       'GEO Region': 'US',
       'Passenger Count': 31353,
       ...}}


- 列に対するさらなる操作

In [15]:
# 最初の5行からの平均乗客数
df["Passenger Count"].head(5).mean()

13565.0

In [16]:
# 上記と同じ操作をクエリ
df_query = "mean:Passenger Count[0..4]"
output = parser.parse(df_query)
print(output)

{'mean': 13565.0}


次の例は```value_counts()```の使用です

In [17]:
# 正しくフォーマットされたクエリ
df_query = "value_counts:GEO Region"
output = parser.parse(df_query)
print(output)

{'value_counts': GEO Region
US                     12475
Asia                    7499
Europe                  5758
Canada                  3216
Mexico                  2651
Australia / Oceania     1712
Central America          809
Middle East              667
South America             91
Name: count, dtype: int64}


- 考察
    - モデルを使用していません

## モデルを使用する場合

実際のLLMと統合してDataFrameに対するクエリを自然言語で実行できます。

In [18]:
# プロンプトテンプレートを作成
prompt = PromptTemplate(
    template="以下の質問に答えてください。\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# チェーンを作成
chain = prompt | model | parser

In [19]:
# 自然言語でクエリを実行
result = chain.invoke({"query": "最初の5行のPassenger Countの平均は?"})
print(result)

{'mean': 13565.0}
